##基礎演習S 後半第3回

NE21-1228F
鈴木　拓己

In [1]:
# 「IPA」フォントをインストール
system("apt-get install -y fonts-ipafont")
# フォント一覧を表示
# systemfonts::system_fonts()
# グラフィクスパラメータのフォントファミリーをIPAGothicとする
par(family="IPAGothic")

In [1]:
if(!require("magrittr")){install.packages("magrittr")}; library(magrittr)
if(!require("rvest")){install.packages("rvest")}; library(rvest)
if(!require("stringr")){install.packages("stringr")}; library(stringr)
if(!require("leaflet")){install.packages("leaflet")}; library(leaflet)
if(!require("htmlwidgets")){install.packages("htmlwidgets")}; library(htmlwidgets)
if(!require("maptools")){install.packages("maptools")}; library(maptools)
if(!require("classInt")){install.packages("classInt")}; library(classInt)
if(!require("RColorBrewer")){install.packages("RColorBrewer")}; library(RColorBrewer)
if(!require("rgdal")){install.packages("rgdal")}; library(rgdal)

Loading required package: magrittr

Loading required package: rvest

Loading required package: stringr

Loading required package: leaflet

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘leaflet’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘lazyeval’, ‘Rcpp’, ‘terra’, ‘gridExtra’, ‘crosstalk’, ‘htmlwidgets’, ‘markdown’, ‘png’, ‘raster’, ‘sp’, ‘viridis’, ‘leaflet.providers’


Loading required package: htmlwidgets

Loading required package: maptools

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘maptools’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: sp

Checking rgeos availability: FALSE
Please note that 'maptools' will be retired by the end of 2023,
plan transition at your earliest co

### Ex. 3-1　地図表示―0, 1次元（点と線）
##### 【問題】 例題3-1と同様にして、東海道五十三次の宿場の名前を地図に（マーカーのマウスオーバーで）ラベル表示し，隣接する宿場を直線で結んで，江戸・日本橋から京・三条大橋への大まかな経路を示しなさい．なお，経度・緯度も（マーカーのマウスクリックで）ポップアップ表示すること．HTMLの地図も提出すること．  
ヒント  
- 例えば，以下からデータを取得して入力すればよい． 
https://ja.wikipedia.org/wiki/東海道五十三次  
-	手入力が嫌ならば，スクレイピングしよう．上記ページの「五十三次の一覧」表の「座標」の「地図」リンク先，あるいは，むしろそのリンクURLの内容から求める･･･ 以下省略（資料を参照のこと）．



###東海道五十三次の宿場の座標

In [2]:
library(rvest)
library(magrittr)

url <- "https://ja.wikipedia.org/wiki/%E6%9D%B1%E6%B5%B7%E9%81%93%E4%BA%94%E5%8D%81%E4%B8%89%E6%AC%A1"
dat <- read_html(url)
#dat
XPath <-"//span/a[@class='external text']"
subj <- html_nodes(dat,xpath=XPath)

spage <- html_attr(subj,"href")
spage <- grep("params", spage, perl=T, value=T)

coordinateN <- gsub("^.*params=([0-9]+(?:\\.?[0-9]+)*)_([0-9]+(?:\\.?[0-9]+)*)_([0-9]+(?:\\.?[0-9]+)*)_N_([0-9]+(?:\\.?[0-9]+)*)_([0-9]+(?:\\.?[0-9]+)*)_([0-9]+(?:\\.?[0-9]+)*).*$","\\1/\\2/\\3",spage,perl=TRUE)

coordinateE <- gsub("^.*params=([0-9]+(?:\\.?[0-9]+)*)_([0-9]+(?:\\.?[0-9]+)*)_([0-9]+(?:\\.?[0-9]+)*)_N_([0-9]+(?:\\.?[0-9]+)*)_([0-9]+(?:\\.?[0-9]+)*)_([0-9]+(?:\\.?[0-9]+)*).*$","\\4/\\5/\\6",spage,perl=TRUE)

coordinateN
coordinateN1 <- gsub("(.*)/(.*)/(.*)","\\1",coordinateN,perl=TRUE); coordinateN1 <- as.numeric(coordinateN1); coordinateN1
coordinateN2 <- gsub("(.*)/(.*)/(.*)","\\2",coordinateN,perl=TRUE); coordinateN2 <- as.numeric(coordinateN2); coordinateN2
coordinateN3 <- gsub("(.*)/(.*)/(.*)","\\3",coordinateN,perl=TRUE); coordinateN3 <- as.numeric(coordinateN3); coordinateN3

coordinateE
coordinateE1 <- gsub("(.*)/(.*)/(.*)","\\1",coordinateE,perl=TRUE); coordinateE1 <- as.numeric(coordinateE1); coordinateE1
coordinateE2 <- gsub("(.*)/(.*)/(.*)","\\2",coordinateE,perl=TRUE); coordinateE2 <- as.numeric(coordinateE2); coordinateE2
coordinateE3 <- gsub("(.*)/(.*)/(.*)","\\3",coordinateE,perl=TRUE); coordinateE3 <- as.numeric(coordinateE3); coordinateE3

lng <- c() # 経度
lat <- c() # 緯度
for(i in 1 : length(coordinateN1)){

  lat_data <- coordinateN1[i] + (coordinateN2[i]/60) + (coordinateN3[i]/60/60)

  lat <- c(lat,lat_data)

  lng_data <- coordinateE1[i] + (coordinateE2[i]/60) + (coordinateE3[i]/60/60)

  lng <- c(lng,lng_data)
}

length(lng); length(lat);

[1] "35/41/01"   "35/37/19"   "35/32/08"   "35/28/22"   "35/26/38.5"
 [6] "35/23/42.1" "35/20/44.4" "35/19/38.2" "35/18/32.4" "35/14/55.4"
[11] "35/11/25.5" "35/7/9.3"   "35/6/0.4"   "35/7/31.9"  "35/9/46"   
[16] "35/7/11.9"  "35/6/27.4"  "35/2/59.4"  "35/1/11.6"  "34/58/29.7"
[21] "34/56/57.9" "34/55/7.8"  "34/52/11.3" "34/49/58.1" "34/49/22.3"
[26] "34/48/14.9" "34/46/22.5" "34/44/50.9" "34/43/36.5" "34/42/18.4"
[31] "34/41/3.8"  "34/41/38.9" "34/41/18.8" "34/43/23"   "34/45/59.9"
[36] "34/50/41.6" "34/51/20.6" "34/54/39.8" "34/57/28.5" "35/0/29.2" 
[41] "35/4/46.8"  "35/07/13.5" "35/4/7.4"   "34/58/10.3" "34/54/15.6"
[46] "34/53/0.2"  "34/51/15.5" "34/51/8.9"  "34/53/18.8" "34/56/3.9" 
[51] "34/57/56"   "35/0/36.2"  "35/1/2.8"   "35/0/21.5"  "35/0/37.2"

[1] 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 34 34 34 34 34 34
[26] 34 34 34 34 34 34 34 34 34 34 34 34 34 34 35 35 35 35 34 34 34 34 34 34 34
[51] 34 35 35 35 35

[1] 41 37 32 28 26 23 20 19 18 14 11  7  6  7  9  7  6  2  1 58 56 55 52 49 49
[26] 48 46 44 43 42 41 41 41 43 45 50 51 54 57  0  4  7  4 58 54 53 51 51 53 56
[51] 57  0  1  0  0

[1]  1.0 19.0  8.0 22.0 38.5 42.1 44.4 38.2 32.4 55.4 25.5  9.3  0.4 31.9 46.0
[16] 11.9 27.4 59.4 11.6 29.7 57.9  7.8 11.3 58.1 22.3 14.9 22.5 50.9 36.5 18.4
[31]  3.8 38.9 18.8 23.0 59.9 41.6 20.6 39.8 28.5 29.2 46.8 13.5  7.4 10.3 15.6
[46]  0.2 15.5  8.9 18.8  3.9 56.0 36.2  2.8 21.5 37.2

[1] "139/46/28"   "139/44/21"   "139/42/28"   "139/37/56.2" "139/35/44"  
 [6] "139/31/47.5" "139/29/10.7" "139/20/16.1" "139/18/55.1" "139/9/39.7" 
[11] "139/1/34.9"  "138/54/52.1" "138/51/26.7" "138/47/50.4" "138/41/8.3" 
[16] "138/36/19.7" "138/34/2.9"  "138/31/10.4" "138/28/51.1" "138/23/15.5"
[21] "138/20/32.8" "138/16/57.6" "138/15/9.8"  "138/10/33.1" "138/7/45.4" 
[26] "138/4/31.3"  "138/0/57.2"  "137/55/23.6" "137/51/25.3" "137/43/54.9"
[31] "137/36/32.2" "137/33/38.8" "137/30/3"    "137/27/52.3" "137/23/23.0"
[36] "137/19/02.4" "137/18/29.6" "137/13/19.9" "137/10/9.1"  "137/2/27.4" 
[41] "136/56/59.3" "136/54/24.2" "136/41/43.8" "136/37/31"   "136/32/52.8"
[46] "136/31/29.4" "136/27/14.8" "136/23/31.5" "136/21/14.7" "136/17/1.8" 
[51] "136/11/0.6"  "136/3/16.7"  "135/57/38.3" "135/51/41.1" "135/46/27.7"

[1] 139 139 139 139 139 139 139 139 139 139 139 138 138 138 138 138 138 138 138
[20] 138 138 138 138 138 138 138 138 137 137 137 137 137 137 137 137 137 137 137
[39] 137 137 136 136 136 136 136 136 136 136 136 136 136 136 135 135 135

[1] 46 44 42 37 35 31 29 20 18  9  1 54 51 47 41 36 34 31 28 23 20 16 15 10  7
[26]  4  0 55 51 43 36 33 30 27 23 19 18 13 10  2 56 54 41 37 32 31 27 23 21 17
[51] 11  3 57 51 46

[1] 28.0 21.0 28.0 56.2 44.0 47.5 10.7 16.1 55.1 39.7 34.9 52.1 26.7 50.4  8.3
[16] 19.7  2.9 10.4 51.1 15.5 32.8 57.6  9.8 33.1 45.4 31.3 57.2 23.6 25.3 54.9
[31] 32.2 38.8  3.0 52.3 23.0  2.4 29.6 19.9  9.1 27.4 59.3 24.2 43.8 31.0 52.8
[46] 29.4 14.8 31.5 14.7  1.8  0.6 16.7 38.3 41.1 27.7

[1] 55

[1] 55

###東海道五十三次の宿場の名前

In [3]:
XPath1 <- "//td[1]/a"
place <- html_nodes(dat,xpath=XPath1)	

place_name <- html_text(place)			
place_name <- place_name[4:59]
#place_name
#place_name[8]
place_name <- place_name[-8]
place_name
length(place_name)

[1] "日本橋"     "品川宿"     "川崎宿"     "神奈川宿"   "保土ヶ谷宿"
 [6] "戸塚宿"     "藤沢宿"     "平塚宿"     "大磯宿"     "小田原宿"  
[11] "箱根宿"     "三島宿"     "沼津宿"     "原宿"       "吉原宿"    
[16] "蒲原宿"     "由比宿"     "興津宿"     "江尻宿"     "府中宿"    
[21] "鞠子宿"     "岡部宿"     "藤枝宿"     "島田宿"     "金谷宿"    
[26] "日坂宿"     "掛川宿"     "袋井宿"     "見付宿"     "浜松宿"    
[31] "舞坂宿"     "新居宿"     "白須賀宿"   "二川宿"     "吉田宿"    
[36] "御油宿"     "赤坂宿"     "藤川宿"     "岡崎宿"     "池鯉鮒宿"  
[41] "鳴海宿"     "宮宿"       "桑名宿"     "四日市宿"   "石薬師宿"  
[46] "庄野宿"     "亀山宿"     "関宿"       "坂下宿"     "土山宿"    
[51] "水口宿"     "石部宿"     "草津宿"     "大津宿"     "三条大橋"

[1] 55

座標の確認

In [4]:
lng
lat

[1] 139.7744 139.7392 139.7078 139.6323 139.5956 139.5299 139.4863 139.3378
 [9] 139.3153 139.1610 139.0264 138.9145 138.8574 138.7973 138.6856 138.6055
[17] 138.5675 138.5196 138.4809 138.3876 138.3424 138.2827 138.2527 138.1759
[25] 138.1293 138.0754 138.0159 137.9232 137.8570 137.7319 137.6089 137.5608
[33] 137.5008 137.4645 137.3897 137.3173 137.3082 137.2222 137.1692 137.0409
[41] 136.9498 136.9067 136.6955 136.6253 136.5480 136.5248 136.4541 136.3921
[49] 136.3541 136.2838 136.1835 136.0546 135.9606 135.8614 135.7744

[1] 35.68361 35.62194 35.53556 35.47278 35.44403 35.39503 35.34567 35.32728
 [9] 35.30900 35.24872 35.19042 35.11925 35.10011 35.12553 35.16278 35.11997
[17] 35.10761 35.04983 35.01989 34.97492 34.94942 34.91883 34.86981 34.83281
[25] 34.82286 34.80414 34.77292 34.74747 34.72681 34.70511 34.68439 34.69414
[33] 34.68856 34.72306 34.76664 34.84489 34.85572 34.91106 34.95792 35.00811
[41] 35.07967 35.12042 35.06872 34.96953 34.90433 34.88339 34.85431 34.85247
[49] 34.88856 34.93442 34.96556 35.01006 35.01744 35.00597 35.01033

###東海道五十三次の宿場の地図

In [5]:
lng		# 経度
lat		# 緯度

pop <- paste(lng, lat, sep=", ")	# ポップアップ

label <- place_name    # ラベル

df <- data.frame(lng, lat, pop)
m <- df %>%
    leaflet %>%
    addTiles %>%
    addMarkers(lng=lng, lat=lat, popup=pop, label=label) %>%
    addPolylines(lng=lng, lat=lat, color="#f30", weight="10")

saveWidget(m, "53_stations.html")

[1] 139.7744 139.7392 139.7078 139.6323 139.5956 139.5299 139.4863 139.3378
 [9] 139.3153 139.1610 139.0264 138.9145 138.8574 138.7973 138.6856 138.6055
[17] 138.5675 138.5196 138.4809 138.3876 138.3424 138.2827 138.2527 138.1759
[25] 138.1293 138.0754 138.0159 137.9232 137.8570 137.7319 137.6089 137.5608
[33] 137.5008 137.4645 137.3897 137.3173 137.3082 137.2222 137.1692 137.0409
[41] 136.9498 136.9067 136.6955 136.6253 136.5480 136.5248 136.4541 136.3921
[49] 136.3541 136.2838 136.1835 136.0546 135.9606 135.8614 135.7744

[1] 35.68361 35.62194 35.53556 35.47278 35.44403 35.39503 35.34567 35.32728
 [9] 35.30900 35.24872 35.19042 35.11925 35.10011 35.12553 35.16278 35.11997
[17] 35.10761 35.04983 35.01989 34.97492 34.94942 34.91883 34.86981 34.83281
[25] 34.82286 34.80414 34.77292 34.74747 34.72681 34.70511 34.68439 34.69414
[33] 34.68856 34.72306 34.76664 34.84489 34.85572 34.91106 34.95792 35.00811
[41] 35.07967 35.12042 35.06872 34.96953 34.90433 34.88339 34.85431 34.85247
[49] 34.88856 34.93442 34.96556 35.01006 35.01744 35.00597 35.01033